In [1]:
import importlib

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

%matplotlib inline

In [2]:
model_module = importlib.import_module("handyrl.model")
env_module = importlib.import_module("handyrl.envs.kaggle.hungry_geese")

Loading environment football failed: No module named 'gfootball'


In [3]:
e = env_module.Environment()
e.reset()

In [4]:
obs = e.observation()
obs = torch.from_numpy(obs.reshape(1, obs.shape[0], obs.shape[1], obs.shape[2])).clone()
obs_ = torch.cat([obs, obs, obs], dim=0)
print(f"size: {obs_.size()}, type: {obs_.dtype}")

size: torch.Size([3, 17, 7, 11]), type: torch.float32


In [5]:
input_ = torch.randn(2, obs.shape[1], obs.shape[2], obs.shape[3])
print(f"size: {input_.size()}, type: {input_.dtype}")

size: torch.Size([2, 17, 7, 11]), type: torch.float32


In [6]:
net = env_module.GeeseNetAlpha()

In [7]:
# Disable trainable for pretrained weights.
# for param in net.offensive.parameters():
#     param.requires_grad = False
# for param in net.defensive.parameters():
#     param.requires_grad = False

In [8]:
out = net(obs_)
# print(f"size: {out.size()}, type: {out.dtype}")
print(f"size: {out['policy'].size()}, type: {out['policy'].dtype}")
# print(f"size: {out['value'].size()}, type: {out['value'].dtype}")

size: torch.Size([3, 4]), type: torch.float32


In [9]:
out

{'policy': tensor([[-0.0208, -0.1368, -0.0292, -0.0725],
         [-0.0208, -0.1368, -0.0292, -0.0725],
         [-0.0208, -0.1368, -0.0292, -0.0725]], grad_fn=<MmBackward>)}

In [10]:
# pytorch_total_params
params = sum(p.numel() for p in net.parameters())
print(f"{params:,}")

568,704


In [11]:
# pytorch_total_params (trainable)
params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"{params:,}")

568,704


In [12]:
net

GeeseNetAlpha(
  (conv0): TorusConv2d(
    (conv): Conv2d(17, 64, kernel_size=(3, 3), stride=(1, 1))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blocks): ModuleList(
    (0): TorusConv2d(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): TorusConv2d(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): TorusConv2d(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): TorusConv2d(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): TorusConv2d(
      (conv): Conv2d(64